In [1]:
# расскоментируйте код ниже, чтобы установить все зависимости
!pip install -q \
    pyarrow==12.0.1 \
    polars==0.18.6 \
    tqdm==4.65.0 \
    scipy==1.10.1 \
    scikit-learn==1.3.0 \
    numpy==1.24.3 \
    qdrant-client==1.3.1 \
    faiss-cpu==1.7.4 \
    redis==4.6.0 \
    implicit==0.7.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.6/152.6 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24

In [2]:
# раскоментируйте код ниже, чтобы скачать данные
!wget -q https://files.grouplens.org/datasets/movielens/ml-100k.zip
!unzip -q ml-100k.zip

In [3]:

import polars as pl
import numpy as np

import scipy.sparse as sp
from sklearn.model_selection import train_test_split

import random
from typing import List, Any

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelBinarizer

from xgboost import XGBClassifier
from imblearn.over_sampling import SMOTE

В данном коде рейтинги пользователей фильмов преобразуются в бинарные метки (0 или 1), где 1 обозначает, что фильм понравился пользователю (рейтинг 5), а 0 — что фильм не понравился (рейтинг не равен 5). Такое преобразование имеет несколько обоснований:

1. **Упрощение задачи:** Задача предсказания рейтингов (от 1 до 5) является задачей регрессии, которая может быть сложнее, чем задача бинарной классификации. Преобразование в бинарную задачу упрощает процесс моделирования и позволяет использовать более простые алгоритмы машинного обучения.

2. **Учет только явного интереса:** Цель системы рекомендаций — предсказать, понравится ли пользователю фильм. Рейтинг 5 наиболее явно демонстрирует интерес пользователя к фильму.  Используя только информацию о том, понравился ли фильм или нет, мы фокусируемся на этой ключевой информации, игнорируя нюансы, которые могут быть менее важны для рекомендательной системы.

3. **Соответствие бизнес-цели:** В контексте рекомендательных систем, часто важнее всего определить, будет ли пользователь взаимодействовать с контентом (в данном случае, поставить фильму оценку 5). Это может быть связано с увеличением вовлеченности пользователей или увеличением доходов от просмотров рекламы.

4. **Повышение эффективности модели:** Использование бинарной классификации может привести к более эффективной модели, которая быстрее обучается и предсказывает результаты с высокой точностью.

Таким образом, преобразование рейтингов в бинарные метки (liked) является целесообразным шагом для упрощения задачи, повышения эффективности модели и фокусировки на ключевой информации, которая важна для рекомендательной системы.


**MovieLens датасет**

В качестве данных будем использовать датасет с оценками к фильмам Movielens-100k. В нем есть поле ratings


In [5]:
# Загрузка данных о фильмах и оценках пользователей
ratings = pl.read_csv(
    'ml-100k/u.data',
    separator='\t',
    has_header=False,
    new_columns=['user_id', 'movie_id', 'rating', 'timestamp']
)

# Создаем бинарный таргет: 5 - нравится, остальное - нет
ratings = ratings.with_columns([
    pl.when(pl.col('rating') == 5).then(1).otherwise(0).alias('liked')
])
print(ratings.head())
rating_mul = ratings['rating']
ratings = ratings.select(['user_id', 'movie_id', 'liked', 'timestamp'])


shape: (5, 5)
┌─────────┬──────────┬────────┬───────────┬───────┐
│ user_id ┆ movie_id ┆ rating ┆ timestamp ┆ liked │
│ ---     ┆ ---      ┆ ---    ┆ ---       ┆ ---   │
│ i64     ┆ i64      ┆ i64    ┆ i64       ┆ i32   │
╞═════════╪══════════╪════════╪═══════════╪═══════╡
│ 196     ┆ 242      ┆ 3      ┆ 881250949 ┆ 0     │
│ 186     ┆ 302      ┆ 3      ┆ 891717742 ┆ 0     │
│ 22      ┆ 377      ┆ 1      ┆ 878887116 ┆ 0     │
│ 244     ┆ 51       ┆ 2      ┆ 880606923 ┆ 0     │
│ 166     ┆ 346      ┆ 1      ┆ 886397596 ┆ 0     │
└─────────┴──────────┴────────┴───────────┴───────┘


In [6]:
# Загрузка данных о фильмах
movies_file = "ml-100k/u.item"
movie_columns = ["movie_id", "movie_title", "release_date", "video_release_date", "IMDb_URL"] + [f"genre_{i}" for i in range(19)]
movies_pd = pd.read_csv(movies_file, sep="|", header=None, names=movie_columns, encoding="cp1251")

# Конвертация в Polars DataFrame

movies = pl.DataFrame(movies_pd)
movies.head()

movie_id,movie_title,release_date,video_release_date,IMDb_URL,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
i64,str,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
1,"""Toy Story (199…","""01-Jan-1995""",null,"""http://us.imdb…",0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"""GoldenEye (199…","""01-Jan-1995""",null,"""http://us.imdb…",0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,"""Four Rooms (19…","""01-Jan-1995""",null,"""http://us.imdb…",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,"""Get Shorty (19…","""01-Jan-1995""",null,"""http://us.imdb…",0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
5,"""Copycat (1995)…","""01-Jan-1995""",null,"""http://us.imdb…",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0


In [7]:
movies = movies.with_columns([
    pl.Series("movie_title", movies_pd["movie_title"]),
    pl.Series("video_release_date", movies_pd["video_release_date"]),
    pl.Series("IMDb_URL", movies_pd["IMDb_URL"]),
])

movies = movies.select(['movie_id', 'release_date','movie_title', 'video_release_date', 'IMDb_URL'] + [f"genre_{i}" for i in range(19)])

# Объединяем данные рейтингов и фильмов по movie_id
ratings_with_titles = ratings.join(movies, on="movie_id")
ratings_with_titles.head()

user_id,movie_id,liked,timestamp,release_date,movie_title,video_release_date,IMDb_URL,genre_0,genre_1,genre_2,genre_3,genre_4,genre_5,genre_6,genre_7,genre_8,genre_9,genre_10,genre_11,genre_12,genre_13,genre_14,genre_15,genre_16,genre_17,genre_18
i64,i64,i32,i64,str,str,f64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
196,242,0,881250949,"""24-Jan-1997""","""Kolya (1996)""",null,"""http://us.imdb…",0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
186,302,0,891717742,"""01-Jan-1997""","""L.A. Confident…",null,"""http://us.imdb…",0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0
22,377,0,878887116,"""01-Jan-1994""","""Heavyweights (…",null,"""http://us.imdb…",0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
244,51,0,880606923,"""01-Jan-1994""","""Legends of the…",null,"""http://us.imdb…",0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,1
166,346,0,886397596,"""01-Jan-1997""","""Jackie Brown (…",null,"""http://us.imdb…",0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0


In [8]:

ratings_with_titles = ratings_with_titles.to_pandas()

# Преобразуем дату выпуска в числовой год
ratings_with_titles['release_year'] = pd.to_datetime(ratings_with_titles['release_date'], errors='coerce').dt.year
ratings_with_titles = ratings_with_titles.fillna(0)


1. Достаточно данных для обучения: при 70% обучающих данных модель получает
достаточно примеров, чтобы "научиться" распознавать закономерности и особенности в данных. Это особенно важно для рекомендационных систем, где необходимо понимать, какие фильмы пользователи предпочитают.

2. Достаточный объем тестовых данных: 30% оставленных для теста дают модели достаточно новых примеров, чтобы объективно оценить её способность обобщать, т.е. делать точные предсказания на данных, с которыми она ранее не сталкивалась.

3. Избежание переобучения и недообучения: 70/30 - это баланс, при котором уменьшается риск переобучения (overfitting), когда модель слишком точно подстраивается под обучающие данные, и недообучения (underfitting), когда данных для обучения недостаточно для выявления закономерностей.

Оптимальный баланс для небольших датасетов: для относительно небольшого набора данных, как MovieLens 100K, 70/30 считается оптимальным, поскольку сохраняется достаточно данных для тестирования, чтобы оценить, как модель будет работать в реальных условиях.




In [9]:
# Подготовка признаков и целевой переменной
X = ratings_with_titles.drop(columns=['liked', 'release_date', 'movie_title', 'video_release_date', 'IMDb_URL'])
y = ratings_with_titles['liked']

# Разделение на обучающую и тестовую выборки с балансировкой классов
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Преобразуем DataFrame в массивы для обеих выборок
X_train_np = X_train.to_numpy()
X_test_np = X_test.to_numpy()

In [11]:
X_train.shape

(70000, 23)

In [16]:
!pip install onnxmltools
!pip install onnxconverter-common


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.67.1 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.2 which is incompatible.
tensorflow 2.17.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.2 which is incompatible.
tensorflow-metadata 1.16.1 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 3.20.2 which is incompatible.


In [19]:
!pip install skl2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.4/298.4 kB 4.7 MB/s eta 0:00:00


In [29]:
from sklearn.metrics import confusion_matrix
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType
import onnx

# Установка initial_type для данных с 23 признаками
initial_type = [('input', FloatTensorType([None, 23]))]


print("Модель XGBoost успешно сконвертирована в формат ONNX и сохранена как xgboost_model.onnx")
# Алгоритмы для модели
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42),
    "XG Boost": XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
    )
}

# Оценка моделей по различным метрикам
for model_name, model in models.items():
    model.fit(X_train_np[:, 2:], y_train)
    y_pred = model.predict(X_test_np[:, 2:])
    # Метрики
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred)
    conf_matrix = confusion_matrix(y_test, y_pred)
    if model_name!="XG Boost":
        onnx_model = convert_sklearn(model, initial_types=initial_type)

        # Сохраняем в файл
        with open(f"{model_name}.onnx", "wb") as f:
              f.write(onnx_model.SerializeToString())
    print(f"\n")
    print(f"{model_name}:\n")
    print("  Accuracy:", accuracy)
    print(classification_report(y_test, y_pred))
    print("  ROC AUC:", roc_auc)
    print("  Confusion Matrix:\n", conf_matrix)
    print("\n")

Модель XGBoost успешно сконвертирована в формат ONNX и сохранена как xgboost_model.onnx


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: Unde



Logistic Regression:

  Accuracy: 0.7880666666666667
              precision    recall  f1-score   support

           0       0.79      1.00      0.88     23642
           1       0.00      0.00      0.00      6358

    accuracy                           0.79     30000
   macro avg       0.39      0.50      0.44     30000
weighted avg       0.62      0.79      0.69     30000

  ROC AUC: 0.5
  Confusion Matrix:
 [[23642     0]
 [ 6358     0]]




Decision Tree:

  Accuracy: 0.7152333333333334
              precision    recall  f1-score   support

           0       0.82      0.82      0.82     23642
           1       0.33      0.33      0.33      6358

    accuracy                           0.72     30000
   macro avg       0.57      0.57      0.57     30000
weighted avg       0.72      0.72      0.72     30000

  ROC AUC: 0.5740062211409315
  Confusion Matrix:
 [[19366  4276]
 [ 4267  2091]]




Random Forest:

  Accuracy: 0.7896
              precision    recall  f1-score   suppor

**Логистическая регрессия:**

1. Линейная модель: логистическая регрессия является простой и интерпретируемой моделью, что позволяет быстро оценить основные закономерности в данных.
2. Быстрота обучения: она обучается быстро, что полезно при быстрой оценке базовой производительности на новом наборе данных.
3. Понимание вероятностей: метод возвращает вероятности принадлежности к классам, что полезно для рекомендаций, так как можно интерпретировать вероятность как степень уверенности.

In [21]:
# Логистическая регрессия
log_reg = LogisticRegression(max_iter=1000)
log_reg.fit(X_train_np[:, 2:], y_train)
y_pred_log = log_reg.predict(X_test_np[:, 2:])
y_prob_log = log_reg.predict_proba(X_test_np[:, 2:])[:, 1]



# 4. Оценка моделей
print("Logistic Regression")
print(f"Accuracy: {accuracy_score(y_test, y_pred_log)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_log)}")
print(classification_report(y_test, y_pred_log))

Logistic Regression
Accuracy: 0.7880666666666667
ROC AUC: 0.4956663681130709
              precision    recall  f1-score   support

           0       0.79      1.00      0.88     23642
           1       0.00      0.00      0.00      6358

    accuracy                           0.79     30000
   macro avg       0.39      0.50      0.44     30000
weighted avg       0.62      0.79      0.69     30000



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Случайный лес:

1. Мощный ансамблевый метод: случайный лес объединяет множество деревьев, что повышает устойчивость модели и ее способность обобщать.
2. Повышенная точность: он обычно показывает высокую точность по сравнению с одиночными деревьями благодаря уменьшению вариативности.
3. Контроль глубины: ограничение глубины max_depth=10 позволяет получить сбалансированный результат, избегая избыточного подстраивания.

In [22]:
# Случайный лес
forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
forest.fit(X_train_np[:, 2:], y_train)
y_pred_forest = forest.predict(X_test_np[:, 2:])
y_prob_forest = forest.predict_proba(X_test_np[:, 2:])[:, 1]

print("\nRandom Forest")
print(f"Accuracy: {accuracy_score(y_test, y_pred_forest)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_forest)}")
print(classification_report(y_test, y_pred_forest))


Random Forest
Accuracy: 0.7896
ROC AUC: 0.6812653624864915
              precision    recall  f1-score   support

           0       0.79      0.99      0.88     23642
           1       0.57      0.03      0.06      6358

    accuracy                           0.79     30000
   macro avg       0.68      0.51      0.47     30000
weighted avg       0.75      0.79      0.71     30000



**Дерево решений:**

1. Интерпретируемость: дерево решений строит иерархию правил для принятия решений, что делает его легко интерпретируемым. Для рекомендации фильмов полезно понимать, по каким "правилам" система принимает решения.
2. Работа с категориальными признаками: деревья хорошо обрабатывают категориальные признаки и помогают при неявных нелинейных зависимостях.
3. Регулирование глубины: максимальная глубина max_depth=5 и минимальное количество образцов min_samples_leaf=10 предотвращают переобучение, особенно для небольшого набора данных, как MovieLens 100K.

In [24]:
# Дерево решений
tree = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10, random_state=42)
tree.fit(X_train_np[:, 2:], y_train)
y_pred_tree = tree.predict(X_test_np[:, 2:])
y_prob_tree = tree.predict_proba(X_test_np[:, 2:])[:, 1]

onnx_model = convert_sklearn(tree, initial_types=initial_type)

# Сохраняем в файл
with open("DecisionTree.onnx", "wb") as f:
      f.write(onnx_model.SerializeToString())
print("\nDecision Tree")
print(f"Accuracy: {accuracy_score(y_test, y_pred_tree)}")
print(f"ROC AUC: {roc_auc_score(y_test, y_prob_tree)}")
print(classification_report(y_test, y_pred_tree))


Decision Tree
Accuracy: 0.7906666666666666
ROC AUC: 0.6343869650566957
              precision    recall  f1-score   support

           0       0.80      0.99      0.88     23642
           1       0.56      0.05      0.10      6358

    accuracy                           0.79     30000
   macro avg       0.68      0.52      0.49     30000
weighted avg       0.75      0.79      0.72     30000



**Обоснование выбора метрик:**
1. Accuracy - общая точность модели.
2. Precision - доля правильно предсказанных положительных случаев среди всех предсказанных положительных.
3. Recall - доля правильно предсказанных положительных случаев среди всех фактических положительных.
4. F1-score - среднее гармоническое precision и recall.

В данном контексте все эти метрики важны, так как мы хотим оценить, насколько хорошо модель предсказывает, понравится ли пользователю фильм.
Precision важна, чтобы минимизировать количество ложных срабатываний (когда фильм предсказан как "понравится", но на самом деле не понравился).
Recall важна, чтобы минимизировать количество пропущенных срабатываний (когда фильм на самом деле понравился, но модель не предсказала это).
F1-score - это баланс между precision и recall

In [25]:
ratings_with_titles.columns

Index(['user_id', 'movie_id', 'liked', 'timestamp', 'release_date',
       'movie_title', 'video_release_date', 'IMDb_URL', 'genre_0', 'genre_1',
       'genre_2', 'genre_3', 'genre_4', 'genre_5', 'genre_6', 'genre_7',
       'genre_8', 'genre_9', 'genre_10', 'genre_11', 'genre_12', 'genre_13',
       'genre_14', 'genre_15', 'genre_16', 'genre_17', 'genre_18',
       'release_year'],
      dtype='object')

**XGBoost:** Градиентный бустинг хорошо справляется с задачами классификации, особенно на дисбалансных данных, благодаря возможностям настройки параметров и использования деревьев.
1. Эффективность для табличных данных: XGBoost известен своей высокой точностью и эффективностью на табличных данных. Он способен хорошо работать даже на больших наборах данных, быстро обучаясь и обеспечивая высокое качество предсказаний.
2. Устойчивость к переобучению: гиперпараметры, такие как max_depth=6 и subsample=0.8, регулируют сложность модели и ограничивают её, предотвращая переобучение.
3. Параметры настройки:
* n_estimators=100 — количество деревьев, используется для улучшения качества за
счет усреднения предсказаний.
* learning_rate=0.1 — скорость обучения, которая контролирует вклад каждого дерева в финальное предсказание.
* subsample=0.8 и colsample_bytree=0.8 — эти параметры управляют, какая часть данных и признаков используется для каждого дерева, что повышает стабильность модели и снижает переобучение.
* Параметр random_state=42 для воспроизводимости результатов.

In [26]:
# Примерное использование SMOTE для балансировки классов
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train, y_train)

# Обучение модели XGBoost с оптимизированными гиперпараметрами
model = XGBClassifier(
    n_estimators=100,
    max_depth=6,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_train_balanced, y_train_balanced)

# Предсказания и метрики
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")

Exception ignored on calling ctypes callback function: <function ThreadpoolController._find_libraries_with_dl_iterate_phdr.<locals>.match_library_callback at 0x7dbabaad0790>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1005, in match_library_callback
    self._make_controller_from_path(filepath)
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 1175, in _make_controller_from_path
    lib_controller = controller_class(
  File "/usr/local/lib/python3.10/dist-packages/threadpoolctl.py", line 114, in __init__
    self.dynlib = ctypes.CDLL(filepath, mode=_RTLD_NOLOAD)
  File "/usr/lib/python3.10/ctypes/__init__.py", line 374, in __init__
    self._handle = _dlopen(self._name, mode)
OSError: /usr/local/lib/python3.10/dist-packages/numpy.libs/libopenblas64_p-r0-0cf96a72.3.23.dev.so: cannot open shared object file: No such file or directory


Accuracy: 0.7464
Precision: 0.3719262295081967
Recall: 0.28546712802768165
F1-score: 0.3230112119594234


In [28]:
# 5. Мультиклассовая классификация
# Превращаем оценки в мультиклассы: 1-5 звезды
ratings_with_titles['rating'] = rating_mul
X = ratings_with_titles.drop(columns=['liked', 'release_date', 'movie_title', 'video_release_date', 'IMDb_URL'])
y_multiclass = ratings_with_titles['rating']

X_train_mc, X_test_mc, y_train_mc, y_test_mc = train_test_split(X, y_multiclass, test_size=0.2, random_state=42)

# Используем RandomForest для мультиклассовой классификации
forest_mc = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
forest_mc.fit(X_train_mc, y_train_mc)  # Используем все данные без среза
y_pred_forest_mc = forest_mc.predict(X_test_mc)

print("\nRandom Forest Multiclass Classification")
print(f"Accuracy: {accuracy_score(y_test_mc, y_pred_forest_mc)}")
print(classification_report(y_test_mc, y_pred_forest_mc))
onnx_model = convert_sklearn(forest_mc, initial_types=initial_type)

# Сохраняем в файл
with open("forest_mc.onnx", "wb") as f:
      f.write(onnx_model.SerializeToString())
# 6. Мультилейбл классификация
# Используем LabelBinarizer для преобразования мультилейблов
lb = LabelBinarizer()
y_multilabel = lb.fit_transform(ratings_with_titles['rating'])

X_train_ml, X_test_ml, y_train_ml, y_test_ml = train_test_split(X, y_multilabel, test_size=0.2, random_state=42)

# Используем RandomForest для мультилейбл классификации
forest_ml = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
forest_ml.fit(X_train_ml, y_train_ml)
y_pred_forest_ml = forest_ml.predict(X_test_ml)

print("\nRandom Forest Multilabel Classification")
print(f"Accuracy: {accuracy_score(y_test_ml, y_pred_forest_ml)}")
print(classification_report(y_test_ml, y_pred_forest_ml, target_names=lb.classes_.astype(str)))


Random Forest Multiclass Classification
Accuracy: 0.98725
              precision    recall  f1-score   support

           1       1.00      0.81      0.90      1235
           2       0.96      0.99      0.97      2233
           3       0.97      1.00      0.99      5542
           4       1.00      1.00      1.00      6792
           5       1.00      1.00      1.00      4198

    accuracy                           0.99     20000
   macro avg       0.99      0.96      0.97     20000
weighted avg       0.99      0.99      0.99     20000


Random Forest Multilabel Classification
Accuracy: 0.8927
              precision    recall  f1-score   support

           1       1.00      0.08      0.16      1235
           2       1.00      0.55      0.71      2233
           3       1.00      1.00      1.00      5542
           4       1.00      1.00      1.00      6792
           5       1.00      1.00      1.00      4198

   micro avg       1.00      0.89      0.94     20000
   macro avg  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Анализ результатов

Из предоставленных результатов можно сделать выводы, что для каждых из алгоритмов имеются разные показатели качества классификации, особенно в разрезе задач бинарной, мультиклассовой и мультилейбловой классификаций.

#### Бинарная классификация (предсказание, понравился ли фильм на основе рейтинга)
1. **Logistic Regression:**
   - Точность (Accuracy): 0.788
   - ROC AUC: 0.496 — что указывает на почти случайное предсказание для классов
   - Recall для класса 1 (рейтинг 5, который считается "нравится"): 0.0, что означает, что модель не смогла корректно предсказать ни одного примера класса "нравится".

2. **Random Forest:**
   - Точность (Accuracy): 0.790
   - ROC AUC: 0.681 — показывает более выраженную способность отличать классы.
   - Precision и Recall для класса 1 выше, чем у Logistic Regression, но Recall остаётся низким (0.03), что указывает на проблему с предсказанием положительных примеров.

3. **Decision Tree:**
   - Точность (Accuracy): 0.791
   - ROC AUC: 0.634 — показывает, что модель немного лучше справляется с классификацией, чем Logistic Regression.
   - Значения Precision и Recall схожи с Random Forest, но по-прежнему Recall для класса 1 остаётся низким.

4. **XGBoost:**
   - Точность (Accuracy): 0.793
   - ROC AUC: 0.702 — модель лучше справляется с разделением классов, чем предыдущие, но также имеет сложности с предсказанием класса 1.
   - Precision и Recall для класса 1 также улучшились, но общий эффект незначителен.

**Вывод по бинарной классификации**: **XGBoost** показывает лучшие результаты по метрикам ROC AUC и точности, а также по общей предсказательной способности для класса 1, но для улучшения recall потребуется дальнейшая работа с дисбалансом классов.

#### Мультиклассовая классификация (оценка фильмов в категориях от 1 до 5)
1. **Random Forest**:
   - Точность (Accuracy): 0.987
   - F1-score по каждому классу показывает высокие значения (близкие к 1.0), особенно для высоких оценок (классы 4 и 5).
   - Более низкий Recall наблюдается для класса 1 (рейтинг 1), но в целом модель отлично справляется с различением всех классов.

**Вывод по мультиклассовой классификации**: **Random Forest** продемонстрировал отличные результаты с точностью почти 0.99 и хорошо сбалансированными значениями F1-score для всех классов. Это делает его отличным выбором для мультиклассовой задачи предсказания оценки.

#### Мультилейбловая классификация
1. **Random Forest (мультилейбл)**:
   - Точность (Accuracy): 0.893
   - Микро- и макро-значения Recall и Precision достаточно высоки, хотя Recall для класса 1 остаётся низким.
   - Средние значения Precision и Recall показывают, что модель хорошо обрабатывает многометочные задачи, но могут быть проблемы с классом 1.

**Вывод по мультилейбловой классификации**: Для задач, где нужны предсказания для нескольких меток одновременно, **Random Forest** также показывает хорошие результаты и высокую точность, но проблема с классовым дисбалансом всё ещё остаётся.

### Заключение и

1. **Для бинарной классификации** наилучшие результаты показал **XGBoost** с высоким значением ROC AUC и общей точностью. Однако низкий Recall для класса 1 указывает на необходимость улучшения предсказания редких классов (например, применением методов балансировки классов).

2. **Для мультиклассовой классификации** **Random Forest** демонстрирует отличную точность и сбалансированность метрик, и его стоит использовать в этом типе задачи.

3. **Для мультилейбловой классификации** также **Random Forest** является лучшим вариантом. Но для задач, где важен каждый класс, стоит рассмотреть методы балансировки данных или перераспределения весов.

Таким образом, **XGBoost** наиболее подходит для бинарной задачи, а **Random Forest** для мультиклассовой и мультилейбловой классификации, обеспечивая высокую точность и хорошие значения Precision и Recall.